In [3]:
import os 
import json
import random
import shutil
import dataset_tools as dtools

In [ ]:
dtools.download(dataset='Cocoa Diseases', dst_dir='~/dataset-ninja/')

#### Box coordinates must be in normalized xywh format (from 0 to 1). If your boxes are in pixels, you should divide x_center and width by image width, and y_center and height by image height. Class numbers should be zero-indexed (start with 0):

In [4]:
annot = "GoodLabels2"
datasetPath = "GoodImages"
def jsonToTxt(pathToAnnots):
    os.makedirs(annot, exist_ok=True)
    for n in os.listdir(pathToAnnots):
        if not n.endswith('.json'):
            continue
        with open(os.path.join(pathToAnnots, n)) as json_file:
            with open(os.path.join(annot, n[:-5] + ".txt"), "w") as txt_file:
                print(n)
                data = json.load(json_file)
                w_img=int(data["size"]["width"])
                h_img=int(data["size"]["height"])
                for nr in data["objects"]:
                    idNum = nr["classId"]
                    points = nr["points"]["exterior"]
                    x_begin=int(points[0][0])
                    y_begin=int(points[0][1])
                    x_end=int(points[1][0])
                    y_end=int(points[1][1])
                    w_box=(x_end-x_begin)/w_img
                    h_box=(y_end-y_begin)/h_img
                    print(f"w_box={w_box} h_box= {h_box}")
                    x_center=x_begin/w_img + w_box/2
                    y_center=y_begin/h_img + h_box/2
                    # write txt file
                    txt_file.write(f"{str(idNum)[-1]} {x_center} {y_center} {w_box} {h_box}\n")
jsonToTxt("../dataset-ninja/cocoa-diseases/all/ann/")

Monilia24.jpg.json
w_box=0.5086538461538461 h_box= 0.4252403846153846
Sana28.jpg.json
w_box=0.1717948717948718 h_box= 0.20120192307692308
w_box=0.06538461538461539 h_box= 0.08966346153846154
w_box=0.08012820512820513 h_box= 0.1362980769230769
w_box=0.04038461538461539 h_box= 0.051201923076923075
w_box=0.05929487179487179 h_box= 0.03485576923076923
w_box=0.04487179487179487 h_box= 0.07860576923076923
w_box=0.14134615384615384 h_box= 0.19014423076923076
w_box=0.058333333333333334 h_box= 0.0798076923076923
w_box=0.04935897435897436 h_box= 0.05889423076923077
w_box=0.1310897435897436 h_box= 0.08245192307692308
w_box=0.08141025641025641 h_box= 0.16682692307692307
w_box=0.09358974358974359 h_box= 0.10048076923076923
Monilia16.jpg.json
w_box=0.22564102564102564 h_box= 0.4543269230769231
w_box=0.08878205128205129 h_box= 0.14014423076923077
w_box=0.0907051282051282 h_box= 0.14759615384615385
w_box=0.09615384615384616 h_box= 0.14543269230769232
w_box=0.05096153846153846 h_box= 0.0875
w_box=0.038

In [5]:

def train_test_split(path,labelPath,outputPath,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")

    #exculde files without jpg extension

    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')])) ## removing duplicate names i.e. counting only number of images

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories
    train_path_img = outputPath + "images/train/"
    train_path_label = outputPath + "labels/train/"
    val_path_img = outputPath + "images/val/"
    val_path_label = outputPath + "labels/val/"


    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)



    ### ----------- copying images to train folder
    for filex in files[:train_size]:
      print(filex)
      if filex == 'classes':
          continue
          
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(labelPath+"/" + filex + '.jpg'+'.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in neg_images:
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in files[train_size:]:
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(labelPath+"/" + filex + '.jpg'+'.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

train_test_split("../dataset-ninja/cocoa-diseases/all/img/",annot,datasetPath)



------ PROCESS STARTED -------
--- This folder has a total number of 312 images---
Monilia51
Fito63
Sana98
Sana21
Sana66
Monilia16
Monilia77
Fito91
Sana11
Sana45
Fito49
Monilia89
Sana13
Sana29
Fito26
Monilia75
Fito3
Monilia67
Monilia33
Fito2
Fito106
Sana32
Fito103
Sana79
Fito6
Monilia63
Fito16
Sana74
Fito100
Sana88
Fito73
Sana80
Fito79
Sana43
Sana16
Fito104
Monilia104
Monilia96
Monilia57
Monilia37
Sana34
Monilia29
Monilia25
Fito38
Fito22
Fito59
Fito85
Monilia99
Sana76
Fito45
Monilia105
Monilia11
Monilia97
Fito67
Monilia12
Fito50
Fito9
Fito32
Monilia23
Monilia7
Sana30
Fito30
Monilia91
Sana94
Monilia78
Sana75
Monilia3
Monilia100
Sana87
Sana97
Monilia76
Monilia17
Monilia49
Fito55
Monilia44
Monilia85
Fito53
Sana99
Sana70
Sana5
Fito78
Fito95
Sana60
Monilia72
Monilia45
Fito43
Sana28
Monilia1
Sana38
Fito23
Fito99
Sana14
Fito4
Monilia59
Fito28
Fito11
Sana84
Fito25
Fito77
Fito18
Sana62
Fito29
Fito69
Sana82
Monilia64
Sana69
Fito102
Monilia103
Sana51
Monilia18
Monilia68
Sana17
Fito70
Sana90
Monil

In [1]:
import torch

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

Number of GPUs available: 8


In [6]:
import torch
from ultralytics import YOLO

# Load the full model
model = YOLO('yolo11n.pt')

# Set the model to evaluation mode

model
# Use the model
train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1],  # GPU
    workers=1  # Minimal number of workers
)

New https://pypi.org/project/ultralytics/8.3.52 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/cocoa_diseases/cocoa_dataset.yaml, epochs=100, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=1, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, so

train: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/train... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:01<00:00, 188.10it/s]
val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val...:   0%|          | 0/62 [00:00<?, ?it/s]

train: New cache created: /home/jovyan/ML2/datasets/cocoa_diseases/labels/train.cache


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<00:00, 173.59it/s]


val: New cache created: /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache
Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         62        289      0.695      0.625      0.675      0.463

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100     0.231G      0.921     0.7691      0.983          7        640: 100%|██████████| 125/125 [00:16<00:00,  7.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  6.93it/s]


                   all         62        289      0.647      0.682      0.678      0.463

100 epochs completed in 0.593 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.48 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:03<00:00,  9.40it/s]


                   all         62        289       0.68      0.648      0.683      0.481
          phytophthora         30         39      0.583      0.564      0.586       0.44
               monilia         23         30      0.806      0.556      0.694       0.53
               healthy         52        220      0.649      0.825      0.767      0.471
Speed: 0.9ms preprocess, 7.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train
